In [2]:
# import csv

# with open('output.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(["Id","Answer"])
#     for i,val in enumerate(ans):
#         writer.writerow([ i, val])

import numpy as np
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import word2vec



C:\Users\a1989\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:

# ans = np.random.randint(6, size=(500, ))
label = np.load('label.npy'); print('label loaded')
cut_ques = np.load('cut_Questions.npy');print('Ques loaded')
model = word2vec.Word2Vec.load("word2vec.model");print('vec loaded')


label loaded
Ques loaded
vec loaded


In [4]:
w1 = "減肥"
w2 = "公斤"
# res = model.wv.most_similar(w1,topn = 10)
# for item in res:
#     print(item[0],str(item[1]))
print(model.wv.similarity(w1,w2))


0.21971726


C:\Users\a1989\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [12]:
import jieba.posseg as pseg
stop_words =  set([' ','這','你','我','的','不','是','是不是','啦','嗎','反而','又','有沒有','耶','啊','這麼','都','了'
                  ,'才','自己','應該'])
time_words = set(['今天','明天','後天','現在'])
def get_score(cut_ques):
    score = np.zeros([6,])
    lines = cut_ques[0]
    d = set()
    for line in lines:
        for word in line:
            if word not in stop_words:
                d.add(word)
    
    for i in range(1, 7):
        line = cut_ques[i]
        for w in line:
            for k in d:
#                 if w not in stop_words:
                try:
                    s = np.power(model.wv.similarity(w,k),4)

                    if (w in time_words) and (k in time_words):
                        if w != k:
                            s = -s
                    if s > 0.1:
                        print(w,k)
                    
                    score[i-1]+= s
                except:
                    #if dict no included, it could be rare name
                    if w == k:
                        score[i-1]+= 3
    return score
get_score(cut_ques[467])


在 在


C:\Users\a1989\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


array([0.15748704, 0.10487895, 1.08455544, 0.06082722, 0.03508933,
       0.14135688])

In [11]:
a = ['a','b','c']
for k in a:
    txt = next(a)
    print(a)

TypeError: 'list' object is not an iterator

In [6]:
import jieba
import jieba.posseg as pseg
jieba.set_dictionary('big5_dict.txt')
example_str = '我討厭吃蘋果'
cut_example_str = jieba.lcut(example_str)
print(cut_example_str) 
words = jieba.posseg.cut("我討厭吃蘋果")
for word, flag in words:
    print('%s %s' % (word, flag))

Building prefix dict from C:\Users\a1989\Desktop\DL\Comp1\big5_dict.txt ...
Loading model from cache C:\Users\a1989\AppData\Local\Temp\jieba.u13ee1a1953b639ea7ba618182ba7d2dd.cache
Loading model cost 0.869 seconds.
Prefix dict has been built succesfully.


['我', '討厭', '吃', '蘋果']
我 N
討厭 Vt
吃 Vt
蘋果 N


In [84]:
import jieba.posseg as pseg
#for each question
ans = np.zeros([500,])
score = np.zeros([500,6])

for q_id,question in enumerate(cut_ques):
    score[q_id,:] = get_score(question)
    ans[q_id] = np.argmax(score[q_id,:])
    
ans = ans.astype(int)
print('done')
a  = (ans==label)
print('accuracy score:',sum(a)/500)

C:\Users\a1989\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


done
accuracy score: 0.622


In [85]:
from termcolor import colored
score = np.round(score,2)
for i in range(500):
    if i in hard:
        print(colored('-------------','green'),i,colored('-------------','green'))
    else:
        print('-------------',i,'-------------')
        
    
    if ans[i]==label[i]:
        pass
#         print(ans[i],label[i],'\t\t',end = '')
#         for si,s in enumerate(score[i,:]):
# #             if si==ans[i]:
# #                 print(colored(s, 'red'),'\t',end = '')
# #             else:
#                 print(s,'\t',end = '')
#         print('v')
    else:
        print(ans[i],label[i],'\t\t',end = '')
        for si,s in enumerate(score[i,:]):
            if si==ans[i]:
                print(colored(s, 'green'),'\t',end = '')
            elif si == label[i]:
                print(colored(s, 'red'),'\t',end = '')
            else:
                print(s,'\t',end = '')
        print()
hard = set([499,494,492,490,486,481,472,471,469])
#477: 排除"你我"不排除"他"

------------- 0 -------------
------------- 1 -------------
------------- 2 -------------
0 3 		1.42 	0.15 	0.06 	1.13 	0.12 	0.52 	
------------- 3 -------------
------------- 4 -------------
------------- 5 -------------
------------- 6 -------------
0 2 		1.39 	0.18 	0.96 	0.15 	0.3 	0.25 	
------------- 7 -------------
0 1 		2.3 	2.05 	0.42 	1.43 	0.16 	1.18 	
------------- 8 -------------
------------- 9 -------------
------------- 10 -------------
1 3 		0.08 	0.18 	0.0 	0.08 	0.11 	0.05 	
------------- 11 -------------
2 5 		1.18 	0.69 	2.05 	0.99 	0.43 	0.54 	
------------- 12 -------------
------------- 13 -------------
------------- 14 -------------
0 3 		0.6 	0.03 	0.37 	0.07 	0.35 	0.56 	
------------- 15 -------------
------------- 16 -------------
------------- 17 -------------
------------- 18 -------------
------------- 19 -------------
------------- 20 -------------
------------- 21 -------------
------------- 22 -------------
3 0 		2.54 	0.07 	0.29 	5.24 	0.39 	0.51 	


------------- 165 -------------
------------- 166 -------------
5 0 		0.01 	0.01 	0.01 	0.0 	0.01 	0.02 	
------------- 167 -------------
------------- 168 -------------
3 1 		1.18 	0.57 	1.04 	1.27 	1.25 	1.2 	
------------- 169 -------------
------------- 170 -------------
------------- 171 -------------
4 3 		1.45 	1.32 	0.46 	0.82 	2.63 	1.26 	
------------- 172 -------------
2 5 		0.1 	0.01 	0.2 	0.03 	0.14 	0.05 	
------------- 173 -------------
------------- 174 -------------
------------- 175 -------------
------------- 176 -------------
------------- 177 -------------
------------- 178 -------------
2 4 		0.17 	0.53 	1.15 	0.37 	0.23 	0.14 	
------------- 179 -------------
------------- 180 -------------
------------- 181 -------------
------------- 182 -------------
------------- 183 -------------
5 2 		0.23 	0.9 	1.0 	0.8 	0.73 	3.01 	
------------- 184 -------------
------------- 185 -------------
4 1 		1.6 	2.05 	2.41 	0.15 	5.48 	0.02 	
------------- 186 -------------
---

1 3 		1.5 	2.24 	1.53 	1.69 	1.22 	1.75 	
------------- 348 -------------
------------- 349 -------------
0 3 		1.14 	0.09 	0.03 	0.26 	0.27 	0.27 	
------------- 350 -------------
4 5 		0.1 	0.15 	0.07 	0.15 	0.21 	0.18 	
------------- 351 -------------
0 3 		1.11 	0.14 	0.3 	0.07 	0.2 	0.16 	
------------- 352 -------------
------------- 353 -------------
------------- 354 -------------
2 0 		0.63 	0.26 	1.28 	0.38 	0.54 	0.51 	
------------- 355 -------------
------------- 356 -------------
------------- 357 -------------
------------- 358 -------------
------------- 359 -------------
0 1 		6.4 	0.46 	2.87 	0.02 	0.97 	0.71 	
------------- 360 -------------
------------- 361 -------------
------------- 362 -------------
------------- 363 -------------
------------- 364 -------------
1 2 		1.34 	2.66 	0.53 	0.75 	0.42 	1.45 	
------------- 365 -------------
0 1 		0.33 	0.25 	0.01 	0.11 	0.02 	0.01 	
------------- 366 -------------
3 4 		0.92 	0.44 	0.05 	1.45 	1.13 	0.34 	
----------

In [80]:
print(cut_ques[468])



[list([['偉傑', '人', '真', '好', ' ', '班', '上', '男生', '應該', '多', '跟', '他', '學學', '才', '對'], []])
 list(['你', '必須', '要', '先', '對', '自己', '誠實', ' ', '你', '才', '有', '辦法', '完整', '的', '跟', '對方', '表達'])
 list(['這樣', '的', '行為', ' ', '已經', '侵犯', '了', '他人', '的', '隱私權', ' ', '是', '要負', '法律', '責任', '的'])
 list(['每個', '作家', '都', '應該', '要', '有', '一台', '筆電', ' ', '才能', '隨時', '隨地', '的', ' ', '記錄下', '自己', '的', '靈感'])
 list(['聽說', '有', '同學', '想要', '人肉', '搜索', '那個', '社群', '網站', '帳號', '的', '主人'])
 list(['居然', '用', '這種', '方式', '贏得', '比賽', ' ', '實在', '勝之不武'])
 list(['對', ' ', '偉傑', '做人', '有', '禮貌', ' ', '服裝', '儀容', '整齊', '又', '乾淨'])]


In [185]:
n = 461
# print(cut_ques[n])
tmp = np.zeros(6,)
# stop_words = [' ','這','你','我','他','的']
for ques in cut_ques[n,0]:
    d = set()
    for w in ques:
        if w not in stop_words:
            d.add(words)
    
    for i in range(6):
        for k in cut_ques[n,i+1]:
#             print(k,w)
            if k in stop_words:
                continue
            try:
                for dd in d:
                    s=np.power(model.wv.similarity(dd,k),4)
            except:pass
#             if s > 0.01:
#                 print(ww,k,s)
            tmp[i]+=s
#             tmp[i]/=len(cut_ques[n,i+1])
print(np.round(tmp,2))


[1.94 1.97 1.28 1.91 3.12 2.57]


C:\Users\a1989\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [43]:
from termcolor import colored
print(colored('hello', 'red'), colored('world', 'green'))
print(colored("hello red world", 'red'))
print('.', end='')
print('.', end='')

hello world
hello red world
..

In [12]:
d = set()
d.add('a')
d.add('b')
for k in d:
    print(k)
g = np.array([4,2,3,1,4])
print(np.argmax(g))

a
b
0


In [14]:
import jieba

jieba.set_dictionary('big5_dict.txt')
example_str = '我討厭吃蘋果'
words = pseg.cut(example_str)


In [75]:
for w in words:
    print('%s %s' % (w.word, w.flag))

Building prefix dict from C:\Users\Administrator\Desktop\DL\Comp1\big5_dict.txt ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.u4db63a6c4a8ce99b1489a4d22431124c.cache
Loading model cost 0.741 seconds.
Prefix dict has been built succesfully.


我 N
討厭 Vt
吃 Vt
蘋果 N


In [20]:
test = np.array([[1,2],[5,4]])
print(np.argmax(test,axis = 1))

[1 0]
